# Orbit Determination 1

This tutorial demonstrates how to determine a satellite orbit using Kalman filtering.

## Setup

In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

In [18]:
from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Interval
from ostk.physics.time import Duration
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Frame
from ostk.physics.coordinate.spherical import LLA
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics.trajectory import Orbit

from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE

from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE

---

## Computation

In [34]:
environment = Environment.default()

Create a Classical Orbital Element (COE) set:

In [35]:
a = Length.kilometers(7000.0)
e = 0.0001
i = Angle.degrees(35.0)
raan = Angle.degrees(40.0)
aop = Angle.degrees(45.0)
nu = Angle.degrees(50.0)

coe = COE(a, e, i, raan, aop, nu)

Setup a Keplerian orbital model:

In [36]:
epoch = Instant.date_time(DateTime(2018, 1, 1, 0, 0, 0), Scale.UTC)
earth = environment.access_celestial_object_with_name("Earth")

keplerian_model = Kepler(coe, epoch, earth, Kepler.PerturbationType.No)

In [37]:
tle = TLE(
    "1 39419U 13066D   18260.77424112  .00000022  00000-0  72885-5 0  9996",
    "2 39419  97.6300 326.6556 0013847 175.2842 184.8495 14.93888428262811"
)

Setup the orbit:

In [38]:
orbit = Orbit(keplerian_model, environment.access_celestial_object_with_name("Earth"))

Now that the orbit is set, we can compute the satellite position:

In [39]:
start_instant = Instant.date_time(DateTime(2018, 9, 5, 0, 0, 0), Scale.UTC)
end_instant = Instant.date_time(DateTime(2018, 9, 6, 0, 0, 0), Scale.UTC)

In [40]:
interval = Interval.closed(start_instant, end_instant)

In [41]:
step = Duration.minutes(1.0)

Generate a time grid:

In [42]:
instants = interval.generate_grid(step)

In [43]:
states = [[instant, orbit.get_state_at(instant)] for instant in instants]

In [44]:
def convert_state (instant, state):
    
    lla = LLA.cartesian(
        state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(),
        Earth.equatorial_radius,
        Earth.flattening
    )
    
    return [
                repr(instant),
                float(instant.get_modified_julian_date(Scale.UTC)),
                *state.get_position().get_coordinates().transpose()[0].tolist(),
                *state.get_velocity().get_coordinates().transpose()[0].tolist(),
                float(lla.get_latitude().in_degrees()),
                float(lla.get_longitude().in_degrees()),
                float(lla.get_altitude().in_meters())
            ]

In [31]:
orbit_data = [convert_state(instant, state) for [instant, state] in states]

In [32]:
orbit_df = pd.DataFrame(orbit_data, columns=['$Time^{UTC}$', '$MJD^{UTC}$', '$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$', '$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$', '$Latitude$', '$Longitude$', '$Altitude$'])

In [33]:
orbit_df.head()

,$Time^{UTC}$,$MJD^{UTC}$,$x_{x}^{ECI}$,$x_{y}^{ECI}$,$x_{z}^{ECI}$,$v_{x}^{ECI}$,$v_{y}^{ECI}$,$v_{z}^{ECI}$,$Latitude$,$Longitude$,$Altitude$
0,2018-09-05 00:00:00 [UTC],58366.000000,2.237807e+06,-5.373364e+06,-3.889422e+06,6585.969931,3521.633618,-1075.275110,-33.872105,-51.142445,629072.508843
1,2018-09-05 00:01:00 [UTC],58366.000694,2.628012e+06,-5.150981e+06,-3.945763e+06,6416.316575,3888.579046,-802.090833,-34.422767,-46.973547,629238.609524
2,2018-09-05 00:02:00 [UTC],58366.001389,3.007229e+06,-4.907060e+06,-3.985606e+06,6219.834159,4239.269980,-525.549955,-34.812566,-42.755088,629347.551276
3,2018-09-05 00:03:00 [UTC],58366.002083,3.373872e+06,-4.642623e+06,-4.008785e+06,5997.343391,4572.240113,-246.808377,-35.037660,-38.503428,629397.187736
4,2018-09-05 00:04:00 [UTC],58366.002778,3.726408e+06,-4.358774e+06,-4.015202e+06,5749.773689,4886.097113,32.968751,-35.095786,-34.236241,629386.330405
